In [2]:
import customtkinter as ctk
from tkinter import filedialog

# 外観モードとテーマの設定（任意）
ctk.set_appearance_mode("light")    # "light"または"dark"
ctk.set_default_color_theme("blue")   # カラーテーマ

# メインウィンドウの作成
app = ctk.CTk()
app.geometry("400x200")
app.title("ディレクトリ選択のデモ")

def select_directory():
    # ディレクトリ選択ダイアログを表示
    directory_path = filedialog.askdirectory(title="ディレクトリを選択してください")
    if directory_path:
        # 選択されたパスをラベルに表示
        label.configure(text=f"選択されたディレクトリ:\n{directory_path}")
    else:
        label.configure(text="ディレクトリが選択されませんでした")

# ボタンウィジェット：クリックでディレクトリ選択ダイアログを表示
select_button = ctk.CTkButton(app, text="ディレクトリを選択", command=select_directory)
select_button.pack(pady=20)

# 選択結果を表示するラベル
label = ctk.CTkLabel(app, text="まだディレクトリが選択されていません")
label.pack(pady=20)

# メインループ開始
app.mainloop()


In [ ]:
%.\realesrgan-ncnn-vulkan.exe -i .\assets\input.jpg -o .\assets\output.jpg -n realesrgan-x4plus